In [ ]:
import pandas as pd
import os
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

## Read data

In [ ]:
from src.features import *
train = load_raw()
cols = imply_columns(train)

In [ ]:
assert not train['Unnamed: 0'].duplicated().any()

In [ ]:
train = train.set_index('Unnamed: 0')

In [ ]:
train.shape, train.columns

In [ ]:
# pd.isnull(train[cols['features']]).sum(axis=1).sort_values(ascending=True).head(n=20)
train.shape[0], len(np.where(pd.isnull(train[cols['features']]).sum(axis=1) < 5)[0])
# train.iloc[0,'1972 [YR1972]']
# pd.isnull(train.loc[0,'1972 [YR1972]'])
# train.head()
# train.loc[131876]

In [ ]:
plt.plot(pd.isnull(train[cols['features']]).sum(axis=1).sort_values(ascending=True).values)
plt.show()

## Plot

In [ ]:
country = 'Kenya' # 'Poland'
# series = list(set(train['Series Name']))
# series.sort()
# series = series[0]
series = 'Net taxes on products (current LCU)'
subtrain = train[(train['Country Name']==country) & (train['Series Name']==series)]
# subtrain.shape
subtrain

In [ ]:
sns.tsplot(subtrain[cols['features']].values.squeeze())
plt.title("%s: %s"%(country, series))
plt.show()
#sns.distplot(train[col])
#plt.title(col)
#plt.show()
#sns.tsplot(np.log(train[col].sort_values()+1))
#plt.title(col)
#plt.show()

## preprocess data to backfill nan

In [ ]:
my_rename = {x: x[6:-1] for x in cols['features']}
train.rename(columns=my_rename, inplace=True)

In [ ]:
cols['features'] = list(my_rename.values())
cols['features'].sort()
cols['features'][:5]

In [ ]:
# wide to long format so that years are an index along the rows
train = pd.wide_to_long(train.reset_index(), ["YR"], i="Unnamed: 0", j="year").reset_index(level='year')

In [ ]:
# rename YR to "value"
train.rename(columns={'YR': 'value'}, inplace=True)

In [ ]:
train['year'] = train['year'].apply(lambda x: int(x))
train['year'].head().values

In [ ]:
# plot again just to confirm that wide-to-long worked
country = 'Kenya' # 'Poland'
series = 'Net taxes on products (current LCU)'
subtrain = train[(train['Country Name']==country) & (train['Series Name']==series)]
subtrain = subtrain.set_index('year')
subtrain.head()

In [ ]:
subtrain['value'].head()

In [ ]:
sns.tsplot(subtrain['value'])
plt.title("%s: %s"%(country, series))
plt.show()

## group and backfill

In [ ]:
train.shape

In [ ]:
# test
df = pd.DataFrame([[np.nan, 2, np.nan, 0, 2, 2],
                   [3, 4, np.nan, 1, 1, 2],
                   [np.nan, np.nan, np.nan, 5, 1, 3],
                   [np.nan, 3, np.nan, 4, 1, 4],
                  ],
                  columns=list('ABCDEF'))
df['B'] = df.sort_values('F', ascending=True).groupby(['E'])['B'].fillna(method='backfill')
df

In [ ]:
train2 = train.copy()

n_nan = pd.isnull(train2['value']).sum()
print('before: number of nan: %s'%n_nan)

train2['value'] = train2.sort_values(['Country Name', 'Series Code', 'year'],
                                     ascending=True
                                    ).groupby(['Country Name', 'Series Code']
                                             )['value'].fillna(method='backfill')

n_nan = pd.isnull(train2['value']).sum()
print('after: number of nan: %s'%n_nan)